Import the libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Load the dataset

In [2]:
df = pd.read_csv("/content/drive/MyDrive/Data/processed_data.csv")
df.head()

,product,subproduct,issue,subissue,narrative,narrative_tokens
0,credit_reporting,Credit reporting,Incorrect information on your report,Information belongs to someone else,hi submitting isnt influence third party trans...,"['hi', 'submitting', 'isnt', 'influence', 'thi..."
1,mortgages_and_loans,Other type of mortgage,Applying for a mortgage or refinancing an exis...,None,applied preapproval property order able view s...,"['applied', 'preapproval', 'property', 'order'..."
2,credit_reporting,Credit reporting,Problem with a credit reporting company's inve...,Problem with personal statement of dispute,investigation found information disputed inacc...,"['investigation', 'found', 'information', 'dis..."
3,credit_reporting,Credit reporting,Improper use of your report,Reporting company used your report improperly,accordance fair credit reporting act violated ...,"['accordance', 'fair', 'credit', 'reporting', ..."
4,debt_collection,Other debt,Attempts to collect debt not owed,Debt is not yours,address credit reporting company tx dear cfpb ...,"['address', 'credit', 'reporting', 'company', ..."


In [3]:
df.shape

(387120, 6)

In [4]:
#No of words column
df['narrative'] = df['narrative'].astype(str)
df['num_words'] = df['narrative'].apply(lambda x: len(x.split()))

In [5]:
#Ecoding the target
encoded_dict  ={'credit_reporting': 0, 'debt_collection': 1, 'mortgages_and_loans': 2,
               'credit_card': 3, 'retail_banking': 4}
df['product'].replace(encoded_dict , inplace=True)
df['product'].unique()

array([0, 2, 1, 4, 3])

In [6]:
# Creating a sample of data
df = df.groupby('product').sample(n=10000)
df = df.reset_index(drop=True)

Load Model and Tokenizer

In [7]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from transformers import AutoTokenizer, TFBertModel

In [8]:
df_train, df_validation = train_test_split(df, test_size=0.15, random_state=42,
                                     stratify=df['product'])
df_train, df_test = train_test_split(df_train, test_size=0.2, random_state=42,
                                     stratify=df_train['product'])

In [9]:
print(df_train.shape)
df_train['product'].value_counts()

(34000, 7)


4    6800
3    6800
0    6800
1    6800
2    6800
Name: product, dtype: int64

In [10]:
print(df_validation.shape)
df_validation['product'].value_counts()

(7500, 7)


1    1500
2    1500
4    1500
0    1500
3    1500
Name: product, dtype: int64

In [11]:
print(df_test.shape)
df_test['product'].value_counts()

(8500, 7)


0    1700
2    1700
4    1700
1    1700
3    1700
Name: product, dtype: int64

In [12]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
df['num_words'].max()

2624

In [23]:
max_len = 128

In [24]:
X_train = tokenizer(
    text=df_train['narrative'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_validation = tokenizer(
    text=df_validation['narrative'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

Model Fitting

In [25]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [26]:
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

In [27]:
embeddings = bert(input_ids, attention_mask = input_mask)[0] # 0 = last hidden state, 1 = poller_output
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(5, activation='softmax')(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [28]:
optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
)

In [29]:
loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

In [30]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

In [31]:
history = model.fit(
    x = {'input_ids':X_train['input_ids'], 'attention_mask':X_train['attention_mask']},
    y = to_categorical(df_train['product']),
    validation_data = ({'input_ids':X_validation['input_ids'], 'attention_mask':X_validation['attention_mask']},
                        to_categorical(df_validation['product'])),
    epochs=1,
    batch_size=32
)

1063/1063 [==============================] - 936s 856ms/step - loss: 0.5864 - balanced_accuracy: 0.8002 - val_loss: 0.4477 - val_balanced_accuracy: 0.8477


Model Evaluation

In [32]:
X_test = tokenizer(
    text=df_test['narrative'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [34]:
#Accuracy on train data
predicted = model.predict({'input_ids': X_train['input_ids'], 'attention_mask': X_train['attention_mask']})
y_predicted = np.argmax(predicted, axis=1)
print(classification_report(df_train['product'], y_predicted))

1063/1063 [==============================] - 283s 267ms/step
              precision    recall  f1-score   support

           0       0.86      0.86      0.86      6800
           1       0.82      0.83      0.83      6800
           2       0.90      0.89      0.89      6800
           3       0.84      0.85      0.84      6800
           4       0.90      0.90      0.90      6800

    accuracy                           0.86     34000
   macro avg       0.86      0.86      0.86     34000
weighted avg       0.86      0.86      0.86     34000



In [33]:
#Accuracy on validation data
from sklearn.metrics import classification_report

predicted = model.predict({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']})
y_predicted = np.argmax(predicted, axis=1)
print(classification_report(df_test['product'], y_predicted))

266/266 [==============================] - 73s 264ms/step
              precision    recall  f1-score   support

           0       0.86      0.85      0.85      1700
           1       0.80      0.80      0.80      1700
           2       0.88      0.87      0.87      1700
           3       0.82      0.82      0.82      1700
           4       0.87      0.89      0.88      1700

    accuracy                           0.85      8500
   macro avg       0.85      0.85      0.85      8500
weighted avg       0.85      0.85      0.85      8500



In [37]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    multiple             108310272   ['input_ids[0][0]',              
                                                                  'attention_mask[0][0]']         
                                                                                                  
 global_max_pooling1d_1 (Global  (None, 768)         0           ['tf_bert_model[1][0]']    

In [39]:
import joblib

In [41]:
joblib.dump(model, '/content/drive/MyDrive/Data/fine_tuned_bert.joblib')

['/content/drive/MyDrive/Data/fine_tuned_bert.joblib']